In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"
N_FOLDS <- 10

In [4]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts030.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 220 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(220): SRR7817611 SRR7817612 ... SRR4317659 SRR4317660
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
grep("ENSMUSG000000032310", rownames(vst.counts))

integer(0)


In [6]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [7]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
# print(vst.count.mtx)

[1] 220


In [8]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec030.Rds", sep = ""))
print(tcdd_dose_detail.vec)

  [1]  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0
 [26]  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0
 [51]  0  0 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0 30 30
 [76] 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0 30  0  0  0  0  0  0
[101]  0  0  0  0  0 30 30 30 30 30 30  0 30 30 30  0  0 30 30 30 30 30 30  0  0
[126]  0  0  0 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30
[151] 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30 30
[176] 30 30 30 30  0  0  0  0 30  0  0  0  0  0  0 30 30 30 30 30  0  0  0  0  0
[201] 30 30 30 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30


In [9]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls030.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[13] "ENSMUSG00000030265" "ENSMUSG00000031074" "ENSMUSG00000031230"
[16] "ENSMUSG00000031603" "ENSMUSG00000036585" "ENSMUSG00000037225"
[19] "ENSMUSG00000043733" "ENSMUSG00000057967"

$`R-MMU-8941623`
 [1] "ENSMUSG00000000182" "ENSMUSG00000000183" "ENSMUSG00000020170"
 [4] "ENSMUSG00000020327" "ENSMUSG00000021732" "ENSMUSG00000021974"
 [7] "ENSMUSG00000022101" "ENSMUSG00000023266" "ENSMUSG00000025219"
[10] "ENSMUSG00000025499" "ENSMUSG00000029337" "ENSMUSG00000030265"
[13] "ENSMUSG00000031074" "ENSMUSG00000031565" "ENSMUSG00000031603"
[16] "ENSMUSG00000036585" "ENSMUSG00000037225" "ENSMUSG00000043733"
[19] "ENSMUSG00000058488"

$`R-MMU-8941628`
 [1] "ENSMUSG00000000182" "ENSMUSG00000020170" "ENSMUSG00000021974"
 [4] "ENSMUSG00000022101" "ENSMUSG00000023266" "ENSMUSG00000025219"
 [7] "ENSMUSG00000025499" "ENSMUSG00000029337" "ENSMUSG00000030265"
[10] "ENSMUSG00000031230" "ENSMUSG00000031603" "ENSMUSG00000036585"
[13] "ENSMUSG00000037225" "ENSMUSG00000043733" "ENSMUSG00000054252"
[16] "E

...

[4] "ENSMUSG00000071042"

$`R-MMU-170986`
[1] "ENSMUSG00000025499" "ENSMUSG00000026821" "ENSMUSG00000030265"

$`R-MMU-5674018`
[1] "ENSMUSG00000025499" "ENSMUSG00000029458" "ENSMUSG00000030265"

$`R-MMU-9647977`
[1] "ENSMUSG00000025499" "ENSMUSG00000030265" "ENSMUSG00000039662"

$`R-MMU-9647980`
[1] "ENSMUSG00000025499" "ENSMUSG00000030265"

$`R-MMU-9649733`
[1] "ENSMUSG00000025499" "ENSMUSG00000030265"

$`R-MMU-9649735`
[1] "ENSMUSG00000025499" "ENSMUSG00000030265"

$`R-MMU-9649736`
[1] "ENSMUSG00000025499" "ENSMUSG00000030265"

$`R-MMU-9658253`
[1] "ENSMUSG00000025499" "ENSMUSG00000027665" "ENSMUSG00000030265"
[4] "ENSMUSG00000031834" "ENSMUSG00000032462" "ENSMUSG00000041417"

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000039153"

$`R-MMU-193401`
[1] "ENSMUSG00000027359"

$`R-MMU-193424`
[1] "ENSMUSG00000027359"

$`R-MMU-193727`
[1] "ENSMUSG00000027359"

$`R-MMU-193743`
[1] "ENSMUSG00000027359"

$`R-MMU-389622`
[1] "ENSMUSG00000027359"

$`R-MMU-389632`
[1] "ENSMUSG00000027359"

$`R-MMU-9734535`
[1] "ENSMUSG00000027359"

$`R-MMU-977301`
[1] "ENSMUSG00000027360"

$`R-MMU-8877153`
[1] "ENSMUSG00000027367"

$`R-MMU-139919`
[1] "ENSMUSG00000027381"

$`R-MMU-573384`
[1] "ENSMUSG00000027395" "ENSMUSG00000028318" "ENSMUSG00000029642"
[4] "ENSMUSG00000036315" "ENSMUSG00000049553" "ENSMUSG00000067148"

$`R-MMU-73747`
[1] "ENSMUSG00000027395" "ENSMUSG00000028318" "ENSMUSG00000029642"
[4] "ENSMUSG00000036315" "ENSMUSG00000049553" "ENSMUSG00000067148"

$`R-MMU-427605`
[1] "ENSMUSG00000027397" "ENSMUSG00000037656"

$`R-MMU-449911`
[1] "ENSMUSG00000027452"

$`R-MMU-176956`
[1] "ENSMUSG00000027454" "ENSMUSG00000031546" "ENSMUSG00000031669"
[4] "ENSMUSG00000031821"

$`R-MMU-6805103`
[1] "ENSMUSG00000027469

 [1] "ENSMUSG00000027869" "ENSMUSG00000027871" "ENSMUSG00000028298"
 [4] "ENSMUSG00000038092" "ENSMUSG00000062410" "ENSMUSG00000063730"
 [7] "ENSMUSG00000090817" "ENSMUSG00000095143" "ENSMUSG00000095388"
[10] "ENSMUSG00000118462"

$`R-MMU-193961`
[1] "ENSMUSG00000027869" "ENSMUSG00000027871" "ENSMUSG00000038092"
[4] "ENSMUSG00000062410" "ENSMUSG00000063730" "ENSMUSG00000090817"
[7] "ENSMUSG00000095143" "ENSMUSG00000095388"

$`R-MMU-196350`
[1] "ENSMUSG00000027869" "ENSMUSG00000027871" "ENSMUSG00000038092"
[4] "ENSMUSG00000062410" "ENSMUSG00000063730" "ENSMUSG00000090817"
[7] "ENSMUSG00000095143" "ENSMUSG00000095388"

$`R-MMU-196372`
[1] "ENSMUSG00000027869" "ENSMUSG00000027871" "ENSMUSG00000038092"
[4] "ENSMUSG00000062410" "ENSMUSG00000063730" "ENSMUSG00000090817"
[7] "ENSMUSG00000095143" "ENSMUSG00000095388"

$`R-MMU-6787811`
[1] "ENSMUSG00000027870"

$`R-MMU-73918`
[1] "ENSMUSG00000027875"

$`R-MMU-2197579`
[1] "ENSMUSG00000027878" "ENSMUSG00000039191" "ENSMUSG00000048001"

$`R-MMU-2

[1] "ENSMUSG00000028790" "ENSMUSG00000038751"

$`R-MMU-8848993`
[1] "ENSMUSG00000028790"

$`R-MMU-110144`
[1] "ENSMUSG00000028792"

$`R-MMU-110145`
[1] "ENSMUSG00000028792"

$`R-MMU-8935732`
[1] "ENSMUSG00000028800" "ENSMUSG00000031622" "ENSMUSG00000031885"
[4] "ENSMUSG00000042557" "ENSMUSG00000049300" "ENSMUSG00000109324"

$`R-MMU-9005995`
[1] "ENSMUSG00000028800" "ENSMUSG00000031393" "ENSMUSG00000042557"

$`R-MMU-9006133`
[1] "ENSMUSG00000028800" "ENSMUSG00000031393" "ENSMUSG00000042557"
[4] "ENSMUSG00000048482"

$`R-MMU-9625102`
[1] "ENSMUSG00000028800" "ENSMUSG00000041798" "ENSMUSG00000042557"
[4] "ENSMUSG00000044167"

$`R-MMU-9625109`
[1] "ENSMUSG00000028800" "ENSMUSG00000041798" "ENSMUSG00000042557"
[4] "ENSMUSG00000044167"

$`R-MMU-8849463`
[1] "ENSMUSG00000028820" "ENSMUSG00000038751"

$`R-NUL-9625758`
[1] "ENSMUSG00000028834"

$`R-NUL-9625772`
[1] "ENSMUSG00000028834"

$`R-MMU-426489`
[1] "ENSMUSG00000028842" "ENSMUSG00000036698" "ENSMUSG00000041530"
[4] "ENSMUSG00000042500" "

In [10]:
%%R
rxns <- rxn2ensembls.nls %>% names()
print(length(rxns))
print(rxns)

[1837] "R-MMU-173512"  "R-MMU-177107"  "R-MMU-2128994" "R-MMU-2160931"
[1841] "R-MMU-2160932" "R-MMU-2160935" "R-MMU-2169046" "R-MMU-2395364"
[1845] "R-MMU-2731078" "R-MMU-442497"  "R-MMU-8870732" "R-MMU-8878117"
[1849] "R-MMU-8937814" "R-MMU-8951966" "R-MMU-9821366" "R-MMU-445448" 
[1853] "R-MMU-5358336" "R-MMU-5358340" "R-MMU-5358343" "R-MMU-5362386"
[1857] "R-MMU-5362412" "R-MMU-5362427" "R-MMU-5362437" "R-MMU-5362441"
[1861] "R-MMU-5362459" "R-MMU-5362549" "R-MMU-5362551" "R-MMU-5362553"
[1865] "R-MMU-5632649" "R-MMU-5632652" "R-MMU-5632654" "R-MMU-9759844"
[1869] "R-MMU-9815762" "R-NUL-5362792" "R-MMU-109638"  "R-MMU-112381" 
[1873] "R-MMU-113402"  "R-MMU-113407"  "R-MMU-113409"  "R-MMU-6803523"
[1877] "R-MMU-72095"   "R-MMU-75095"   "R-MMU-77095"   "R-MMU-9012315"
[1881] "R-MMU-9012319" "R-MMU-9838035" "R-MMU-9838289" "R-MMU-8853734"
[1885] "R-MMU-8853737" "R-MMU-8853753" "R-MMU-8853755" "R-MMU-8853762"
[1889] "R-MMU-8853774" "R-MMU-8853792" "R-MMU-8853793" "R-MMU-8853801"
[1893]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "R-MMU-1454713" "R-MMU-2028644" "R-MMU-392513" 
[4553] "R-MMU-450406"  "R-MMU-482788"  "R-MMU-5632738" "R-MMU-75016"  
[4557] "R-MMU-913993"  "R-MMU-9614423" "R-MMU-3341397" "R-MMU-5332993"
[4561] "R-MMU-5684006" "R-MMU-5684008" "R-MMU-9619130" "R-MMU-9619142"
[4565] "R-MMU-178208"  "R-MMU-201445"  "R-MMU-201821"  "R-MMU-2176445"
[4569] "R-MMU-2179274" "R-MMU-5632642" "R-MMU-5635865" "R-MMU-984713" 
[4573] "R-MMU-984733"  "R-MMU-1675795" "R-MMU-1676065" "R-MMU-1676105"
[4577] "R-MMU-1676141" "R-MMU-2187401" "R-MMU-2187405" "R-MMU-201629" 
[4581] "R-NUL-8951809" "R-MMU-158218"  "R-MMU-158251"  "R-MMU-158311" 
[4585] "R-MMU-158313"  "R-MMU-158354"  "R-MMU-433101"  "R-MMU-8943780"
[4589] "R-MMU-9679787" "R-MMU-4754187" "R-MMU-5661121" "R-MMU-8979071"
[4593] "R-MMU-9832372" "R-MMU-9833358" "R-MMU-9833367" "R-MMU-418925" 
[4597] "R-MMU-5684862" "R-MMU-8954082" "R-MMU-1675776" "R-MMU-1676145"
[4601] "R-MMU-6811522" "R-MMU-8877691" "R-MMU-8877692" "R-MMU-4419948"
[4605] "R-MMU-1449573" "R-MM

[5469] "R-MMU-9823983" "R-MMU-209055"  "R-MMU-2187388" "R-MMU-8878815"
[5473] "R-MMU-187949"  "R-MMU-425861"  "R-MMU-1234164" "R-MMU-1234177"
[5477] "R-MMU-1234181" "R-MMU-72231"   "R-MMU-77591"   "R-MMU-77592"  
[5481] "R-MMU-77593"   "R-MMU-2995389" "R-MMU-381111"  "R-MMU-5227009"
[5485] "R-MMU-72663"   "R-MMU-72669"   "R-MMU-9635380" "R-MMU-9653710"
[5489] "R-MMU-9653771" "R-MMU-9653773" "R-MMU-9655565" "R-MMU-9840415"
[5493] "R-MMU-2464822" "R-MMU-452036"  "R-MMU-450488"  "R-MMU-450517" 
[5497] "R-MMU-194121"  "R-MMU-191382"  "R-MMU-8857662" "R-MMU-5693578"
[5501] "R-MMU-198669"  "R-MMU-198756"  "R-MMU-199910"  "R-MMU-199935" 
[5505] "R-NUL-5368587" "R-MMU-1592387" "R-MMU-2129353" "R-NUL-2426460"
[5509] "R-MMU-8847875" "R-MMU-5688834" "R-MMU-5689085" "R-MMU-9617927"
[5513] "R-MMU-1678920" "R-MMU-2130336" "R-MMU-6808566" "R-MMU-192036" 
[5517] "R-MMU-192160"  "R-MMU-193758"  "R-MMU-193781"  "R-MMU-193800" 
[5521] "R-MMU-193841"  "R-MMU-2161549" "R-MMU-2161614" "R-MMU-2855252"
[5525]

[6389] "R-MMU-9702818" "R-MMU-9702849" "R-MMU-2586748" "R-MMU-210300" 
[6393] "R-MMU-416725"  "R-MMU-5694071" "R-MMU-266082"  "R-MMU-3257122"
[6397] "R-MMU-1614524" "R-MMU-74247"   "R-MMU-74258"   "R-MMU-9727347"
[6401] "R-MMU-9727349" "R-MMU-9748991" "R-MMU-9833635" "R-MMU-9833777"
[6405] "R-MMU-9834807" "R-MMU-9835121" "R-MMU-9835189" "R-MMU-9835328"
[6409] "R-MMU-9835385" "R-MMU-9835944" "R-MMU-9836159" "R-MMU-9836362"
[6413] "R-MMU-9836435" "R-MMU-9836515" "R-MMU-9836606" "R-MMU-9836617"
[6417] "R-MMU-9836702" "R-MMU-975814"  "R-MMU-2161795" "R-MMU-2161814"
[6421] "R-MMU-2161890" "R-MMU-2161899" "R-MMU-2161940" "R-MMU-6788556"
[6425] "R-MMU-391377"  "R-MMU-6794346" "R-MMU-6797974" "R-MMU-109701" 
[6429] "R-MMU-202164"  "R-MMU-202222"  "R-MMU-202394"  "R-MMU-202437" 
[6433] "R-MMU-202443"  "R-MMU-202459"  "R-MMU-202466"  "R-MMU-202472" 
[6437] "R-MMU-202478"  "R-MMU-202489"  "R-MMU-2219524" "R-MMU-2316429"
[6441] "R-MMU-9632858" "R-MMU-9632906" "R-MMU-9632910" "R-MMU-9632918"
[6445]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "R-MMU-9839110"
[8853] "R-MMU-6813626" "R-MMU-1482887" "R-MMU-429101"  "R-MMU-2213239"
[8857] "R-MMU-417829"  "R-MMU-9611277" "R-MMU-417843"  "R-MMU-417858" 
[8861] "R-MMU-70471"   "R-MMU-70475"   "R-MMU-6781899" "R-NUL-9620900"
[8865] "R-MMU-1679098" "R-MMU-187895"  "R-MMU-1168635" "R-MMU-9605682"
[8869] "R-MMU-5692480" "R-MMU-446214"  "R-MMU-2730959" "R-MMU-2395818"
[8873] "R-MMU-2395849" "R-MMU-2395872" "R-MMU-9731228" "R-MMU-9731590"
[8877] "R-MMU-9731613" "R-MMU-9731632" "R-MMU-426520"  "R-MMU-5578712"
[8881] "R-MMU-5578965" "R-MMU-5578966" "R-MMU-9023912" "R-MMU-9820843"
[8885] "R-MMU-9820863" "R-MMU-1299318" "R-MMU-6786720" "R-MMU-8851538"
[8889] "R-MMU-8851550" "R-MMU-352052"  "R-MMU-6803789" "R-MMU-8847904"
[8893] "R-MMU-9017129" "R-MMU-8874745" "R-MMU-173626"  "R-MMU-173631" 
[8897] "R-MMU-6784224" "R-MMU-8955817" "R-NUL-1458871" "R-MMU-1678921"
[8901] "R-MMU-1678998" "R-MMU-1461993" "R-MMU-1602488" "R-MMU-3214912"
[8905] "R-MMU-5661123" "R-MMU-8849908" "R-MMU-8938887" "R-MM

[9773] "R-MMU-5619422" "R-MMU-446195"  "R-MMU-9018868" "R-MMU-9018895"
[9777] "R-MMU-9020271" "R-MMU-9020273" "R-MMU-350869"  "R-MMU-5696004"
[9781] "R-MMU-9670578" "R-MMU-8955010" "R-MMU-71825"   "R-MMU-5696146"
[9785] "R-MMU-6788810" "R-MMU-8944246" "R-MMU-420384"  "R-MMU-6786239"
[9789] "R-MMU-9717434" "R-MMU-5619427" "R-MMU-1605723" "R-MMU-4085029"
[9793] "R-MMU-8878787" "R-MMU-162821"  "R-MMU-5619433" "R-MMU-6799977"
[9797] "R-MMU-6799981" "R-MMU-174916"  "R-MMU-8941701" "R-MMU-904864" 
[9801] "R-MMU-429663"  "R-MMU-8954327" "R-MMU-159179"  "R-MMU-159194" 
[9805] "R-MMU-9756134" "R-MMU-9756156" "R-MMU-9756183" "R-MMU-9632038"
[9809] "R-MMU-9632039" "R-MMU-5695989" "R-MMU-209821"  "R-MMU-191323" 
[9813] "R-MMU-159790"  "R-MMU-9026967" "R-MMU-5619415" "R-MMU-6807826"
[9817] "R-MMU-1655879" "R-MMU-444419"  "R-MMU-446278"  "R-MMU-6786789"
[9821] "R-MMU-8854736" "R-MMU-9604767" "R-MMU-9706364" "R-MMU-8850818"
[9825] "R-MMU-450971" 


In [11]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [12]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220


In [13]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

  [1]  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0
 [26]  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0
 [51]  0  0 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0 30 30
 [76] 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0 30  0  0  0  0  0  0
[101]  0  0  0  0  0 30 30 30 30 30 30  0 30 30 30  0  0 30 30 30 30 30 30  0  0
[126]  0  0  0 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30
[151] 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30  0  0  0  0  0  0 30 30
[176] 30 30 30 30  0  0  0  0 30  0  0  0  0  0  0 30 30 30 30 30  0  0  0  0  0
[201] 30 30 30 30 30 30 30 30  0  0  0  0  0  0 30 30 30 30 30 30


In [14]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001  19.253048  19.284482  19.227401  19.303631  19.270726
ENSMUSG00000000028  11.615180  11.441836  11.753709  11.555786  11.642012
ENSMUSG00000000049  23.731170  23.717182  23.973936  23.931979  23.934691
ENSMUSG00000000058  12.370309  12.555203  12.607501  12.228821  12.921381
ENSMUSG00000000085  14.694152  14.372059  14.591633  14.574837  14.096266
ENSMUSG00000000088  20.435288  20.448621  20.427604  20.079115  20.025559
ENSMUSG00000000120  13.804337  13.893935  13.975519  13.438152  13.355111
ENSMUSG00000000126  11.327511  11.757773  11.184232  11.820398  11.631200
ENSMUSG00000000127  13.730438  13.476003  13.879914  13.730664  14.073197
ENSMUSG00000000142  13.971671  13.951535  13.938735  14.949457  14.616531
ENSMUSG00000000149  19.555741  19.645196  19.623362  19.737432  19.487447
ENSMUSG00000000154  18.948658  19.063887  19.170692  19.216970  19.231479
ENSMUSG00000000168  18.077533  18.1214

ENSMUSG00000002249  14.843946  14.879820  15.058250  15.137849  14.790433
ENSMUSG00000002250  17.874471  18.040800  17.449484  18.262208  18.352773
ENSMUSG00000002257  11.129961  10.991789  11.065028  11.409958  11.323557
ENSMUSG00000002279  16.735295  17.223670  17.224379  17.440394  17.308679
ENSMUSG00000002289  16.202008  16.009790  17.560282  15.913124  16.203094
ENSMUSG00000002297  12.420497  12.610172  12.225467  12.132897  11.973377
ENSMUSG00000002307  15.073966  15.100359  15.173345  15.299765  15.296376
ENSMUSG00000002308  13.373876  13.338179  12.948802  13.662272  13.260268
ENSMUSG00000002324  12.263154  11.972283  12.786687  11.560118  11.817556
ENSMUSG00000002325  17.199320  17.314219  17.242617  17.615599  17.766004
ENSMUSG00000002326  15.394773  15.434568  15.137617  15.199471  15.338526
ENSMUSG00000002346  19.595184  19.640911  19.493834  19.538865  19.574738
ENSMUSG00000002365  17.252034  17.311938  17.675594  17.614394  17.685699
ENSMUSG00000002379  17.847160  17.6090

ENSMUSG00000004654   9.843375   9.842703   9.827793   9.842897   9.842776
ENSMUSG00000004655  18.698115  18.698493  18.425005  18.672660  18.667431
ENSMUSG00000004665  13.778591  13.475495  13.404278  13.678734  14.098249
ENSMUSG00000004667  16.812564  17.027787  16.631326  16.443199  16.589643
ENSMUSG00000004668   9.843375  10.641375   9.827793   9.842897   9.842776
ENSMUSG00000004677  17.277952  17.290170  16.829205  16.992289  16.905217
ENSMUSG00000004709  10.644593  11.124534  10.761627  10.642307  11.210171
ENSMUSG00000004730  16.011591  16.074114  13.929133  15.384577  15.345917
ENSMUSG00000004768  14.521483  14.178081  14.562445  14.445464  14.483215
ENSMUSG00000004771  18.139888  18.283834  18.225789  18.148326  18.059334
ENSMUSG00000004788  15.994080  16.220256  15.827473  15.837129  15.874136
ENSMUSG00000004789  20.776441  20.800125  20.713025  20.740338  20.811666
ENSMUSG00000004791  10.644593  10.641375   9.827793   9.842897   9.842776
ENSMUSG00000004815  17.547509  17.4791

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




ENSMUSG00000001999  15.323664  15.242687  14.931197  14.523591  14.921311
ENSMUSG00000002010  18.909277  18.326534  18.452031  19.081285  19.026221
ENSMUSG00000002015  19.649382  19.750024  19.783425  19.597894  19.634833
ENSMUSG00000002020  15.506443  11.386184  11.671636  11.757988  11.379797
ENSMUSG00000002028  16.065472  16.744323  16.794034  16.968502  16.790313
ENSMUSG00000002031  15.626126  16.411860  16.329538  16.292123  16.380750
ENSMUSG00000002033  11.222982  10.989182  10.996625  10.894567  11.364746
ENSMUSG00000002043  15.024251  14.391863  14.519917  14.407787  14.356354
ENSMUSG00000002052  17.599733  17.610388  17.692863  17.854896  17.800022
ENSMUSG00000002059  13.903712  13.401566  13.131853  13.412225  13.377460
ENSMUSG00000002068  13.336706  10.999954  10.637416  11.389077  11.379797
ENSMUSG00000002100  10.893811   9.842337  10.637416   9.849511  11.172350
ENSMUSG00000002102  19.340490  19.612012  19.699080  19.666657  19.563349
ENSMUSG00000002108  17.543685  19.337

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  19.228616  19.261461  19.265421
ENSMUSG00000002963  16.710357  16.737745  16.730076  16.697816  16.848683
ENSMUSG00000002968  17.837364  17.549733  17.640628  17.524284  17.353804
ENSMUSG00000002983  12.468127  11.879400  12.258093  11.920848  12.196518
ENSMUSG00000002984  16.408939  17.566054  17.505922  16.415910  16.488424
ENSMUSG00000002985  26.893469  27.317748  27.327144  27.565844  27.558534
ENSMUSG00000002992  19.938056  20.467157  20.370782  20.568920  20.581061
ENSMUSG00000003031  16.883381  17.220476  17.166874  17.026674  17.071145
ENSMUSG00000003032  11.504562  12.101096  12.326203  11.568365  11.868667
ENSMUSG00000003033  17.128928  16.881726  17.102776  17.050945  17.265917
ENSMUSG00000003037  18.577720  18.948547  18.925468  18.668477  18.490733
ENSMUSG00000003039  17.248876  17.698662  17.662512  17.538974  17.927393
ENSMUSG00000003051  11.406050  11.268068  11.270581  11.440281  11.303357
ENSMUSG00000003053  23.279524  23.347509  23.353317  23.540663  23.525833
ENSM

ENSMUSG00000000555  15.938554  15.960206  15.700905  15.572961   15.64094
ENSMUSG00000000560   9.819933   9.856386   9.807625   9.814679    9.85135
ENSMUSG00000000561  16.483658  16.433272  16.620482  16.767804   16.68987
ENSMUSG00000000562   9.819933   9.856386  10.645509  10.906732    9.85135
ENSMUSG00000000563  20.726327  20.594721  20.689999  20.441992   20.56695
ENSMUSG00000000567  13.755802  13.792814  13.529842  12.673498   12.50933
ENSMUSG00000000568  18.339209  18.201048  18.279036  17.813675   17.72456
ENSMUSG00000000579  11.903767  11.660222  11.800276  11.400739   11.66289
ENSMUSG00000000581  17.702939  17.841355  17.442384  17.593058   17.62213
ENSMUSG00000000594  18.697802  18.669302  18.748859  18.782522   18.92971
ENSMUSG00000000605  18.165779  17.991186  18.070219  18.111900   18.20162
ENSMUSG00000000617  11.553564  11.935580  11.520612  10.513360    9.85135
ENSMUSG00000000627  10.536434  10.708179  10.839614  10.906732   10.90620
ENSMUSG00000000628  13.608123  13.5481

ENSMUSG00000002797  14.424664  14.413567  14.420258  13.930984   13.99175
ENSMUSG00000002799  11.480003  11.262139  11.260477  11.308855   11.06574
ENSMUSG00000002803  14.483810  14.145124  14.221729  14.685295   14.66110
ENSMUSG00000002804  14.514289  14.454362  14.546942  15.024579   15.15114
ENSMUSG00000002814  13.857824  14.029619  13.657480  13.762700   13.75896
ENSMUSG00000002820  16.382283  16.531754  16.518094  17.113714   17.25750
ENSMUSG00000002846  15.962563  15.756162  16.027011  16.161246   16.08180
ENSMUSG00000002847  13.842772  14.141169  14.055534  14.067227   14.30559
ENSMUSG00000002870  14.362449  14.022402  14.169819  14.298685   14.01898
ENSMUSG00000002885  16.454715  15.882420  16.191927  16.379095   16.39811
ENSMUSG00000002900  14.931889  14.901116  14.847820  14.066992   13.87426
ENSMUSG00000002908  12.569462  12.458057  12.362704  13.409557   13.34509
ENSMUSG00000002944  21.879401  21.838231  21.866916  21.495919   21.48073
ENSMUSG00000002948  16.984334  17.2389

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

In [16]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)
print(length(vst.count.mtx.test))

  18.697396  18.944585
ENSMUSG00000001247  19.343726  19.300854   19.17694  20.063814  20.107541
ENSMUSG00000001249  20.675860  20.769989   20.71199  21.829709  21.800227
ENSMUSG00000001270  15.219846  15.143396   15.09572  12.420160  12.560676
ENSMUSG00000001280  16.931977  16.613009   17.03898  17.875024  17.862511
ENSMUSG00000001281  14.965584  13.786312   13.64193  11.823256  11.232611
ENSMUSG00000001288  14.584506  14.436884   13.99985  13.167919  13.149050
ENSMUSG00000001300  13.187276  13.517188   13.55982  12.476723  12.179892
ENSMUSG00000001313  15.470638  14.673922   14.49593  14.843911  14.733034
ENSMUSG00000001323  18.988984  19.184019   19.22552  19.863698  19.937922
ENSMUSG00000001348  19.167847  18.635228   18.64214  19.611968  19.671892
ENSMUSG00000001366  17.275211  17.619461   17.47893  18.602405  18.852759
ENSMUSG00000001383  16.627534  16.854835   16.66363  17.199027  17.169982
ENSMUSG00000001403  12.788200  12.290421   12.43422  10.961510  11.126229
ENSMUSG00000001

ENSMUSG00000003484  14.261301  13.083034   12.94743  11.099643  11.161435
ENSMUSG00000003500  15.063049  15.181615   15.11127  12.240158  12.500717
ENSMUSG00000003505   9.816692   9.819421    9.81925   9.859964   9.859415
ENSMUSG00000003518  18.259317  18.277115   18.26205  17.091628  16.998660
ENSMUSG00000003526  17.922059  17.613890   17.68849  20.983308  20.864293
ENSMUSG00000003528  19.452825  19.521552   19.39339  20.902497  21.032655
ENSMUSG00000003534  15.405077  14.953885   14.72352  12.061582  11.837977
ENSMUSG00000003541  14.858924  13.454961   13.54243  11.758562  11.761642
ENSMUSG00000003546  17.250163  17.065186   17.00952  18.726624  18.595237
ENSMUSG00000003549  12.924287  13.004764   13.01272  12.889523  12.585981
ENSMUSG00000003555  11.327102  11.549915   11.59430  13.592443  13.530265
ENSMUSG00000003559  14.656712  14.066917   14.62256  16.877483  17.044323
ENSMUSG00000003573  14.919252  14.806924   14.66861  14.197337  14.071948
ENSMUSG00000003604  14.337926  14.2991

ENSMUSG00000001105   16.66516  16.674561  17.085339  16.821954   15.62548
ENSMUSG00000001119   13.27155  12.984135  12.801416  12.650418   15.94501
ENSMUSG00000001123   22.06058  22.082071  21.997466  22.020634   19.74480
ENSMUSG00000001127   19.07934  19.023612  18.788596  18.635342   18.40855
ENSMUSG00000001128   14.43019  14.612428  14.860933  14.556826   16.62143
ENSMUSG00000001131   10.79141  10.625271  11.244786  10.951481   12.94991
ENSMUSG00000001143   15.08430  14.679796  15.194416  14.764819   13.97761
ENSMUSG00000001155   22.57786  22.527185  22.404352  22.376048   18.54087
ENSMUSG00000001158   15.12759  14.870950  14.810779  15.086078   14.61920
ENSMUSG00000001173   14.23546  14.437729  14.235770  13.834534   14.83725
ENSMUSG00000001175   20.88289  20.833640  20.724219  20.821402   19.89170
ENSMUSG00000001211   20.69623  20.661992  20.578001  20.634826   20.73216
ENSMUSG00000001225    9.83910   9.839319   9.902304   9.902349   10.69670
ENSMUSG00000001228   11.28859  11.3734

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



   16.95845  16.787120  18.260401  18.335579
ENSMUSG00000003378  14.371463   15.33619  15.408091  14.562057  14.241644
ENSMUSG00000003379  11.974572   11.86880  12.057134  12.418478  11.937725
ENSMUSG00000003402  19.271125   19.89765  20.034532  19.499186  19.416741
ENSMUSG00000003411  12.915372   12.12370  12.424891  12.575862  12.661213
ENSMUSG00000003418  11.153284    9.83301  10.931195  11.123756  11.120237
ENSMUSG00000003421  15.679978   15.66715  15.646766  15.852717  15.692095
ENSMUSG00000003429  17.773402   18.40565  18.245017  18.629438  18.566232
ENSMUSG00000003435  18.233272   17.83424  17.780408  18.222729  18.304832
ENSMUSG00000003436   9.816511    9.83301   9.832642   9.830479   9.830276
ENSMUSG00000003437  16.519321   17.23894  17.249601  16.580918  16.737598
ENSMUSG00000003444  12.669897   12.69614  12.412638  12.852696  12.920304
ENSMUSG00000003452  11.480260   11.13790  11.044737  11.485693  11.279183
ENSMUSG00000003458  17.839807   17.47936  17.575839  18.256028  18.

...


ENSMUSG00000001518  15.139452  14.341273  14.798087   14.87532  14.539578
ENSMUSG00000001521  12.114124  12.275377  12.188112   11.64564  12.022085
ENSMUSG00000001524  15.315382  15.148237  15.566365   15.13224  15.487998
ENSMUSG00000001525  19.676192  19.667088  19.549933   19.36020  19.451348
ENSMUSG00000001542  17.431306  17.579331  17.542609   17.51246  17.623135
ENSMUSG00000001552  16.596632  16.509377  16.027089   16.05898  16.981225
ENSMUSG00000001622   9.847413   9.847641   9.899352    9.90006   9.857265
ENSMUSG00000001632  15.186045  15.279027  15.154071   14.90885  15.168166
ENSMUSG00000001642  13.005821  12.770300  12.780300   12.95677  12.016792
ENSMUSG00000001656   9.847413   9.847641   9.899352    9.90006   9.857265
ENSMUSG00000001657   9.847413   9.847641   9.899352    9.90006   9.857265
ENSMUSG00000001663  21.097164  21.127433  20.761968   20.90592  21.116408
ENSMUSG00000001670  23.736680  23.741086  23.360133   23.34712  23.534393
ENSMUSG00000001729  17.779514  17.673

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  19.608916  19.565599   19.78152  19.643139
ENSMUSG00000001855  15.003401  15.599285  15.497535   15.05384  15.045533
ENSMUSG00000001865   9.857861   9.866686   9.866651    9.85394   9.864916
ENSMUSG00000001870  12.780912  12.411665  12.346931   13.62718  12.810589
ENSMUSG00000001891  20.952478  21.295644  21.337612   20.61038  21.041340
ENSMUSG00000001901  10.853086   9.866686   9.866651    9.85394   9.864916
ENSMUSG00000001918  12.991409  12.758242  12.712019   15.00396  12.927586
ENSMUSG00000001924  20.461088  20.879568  20.821113   20.57684  20.636241
ENSMUSG00000001930  13.470769  14.284018  14.437181   14.48176  13.480025
ENSMUSG00000001946  14.494968  14.038480  14.036903   15.67215  13.838185
ENSMUSG00000001962  15.839362  14.717190  15.084598   15.70578  15.831271
ENSMUSG00000001964  15.759740  15.683685  15.410030   15.80506  15.664546
ENSMUSG00000001983  14.594382  15.085313  15.387203   14.72794  14.939380
ENSMUSG00000001985   9.857861   9.866686   9.866651    9.85394   9.

ENSMUSG00000004221  19.572362  19.826950  19.691952   19.42715  19.655995
ENSMUSG00000004231   9.857861   9.866686   9.866651    9.85394   9.864916
ENSMUSG00000004263  15.213234  15.280834  15.291110   15.19420  15.675512
ENSMUSG00000004264  19.955341  19.534713  19.592575   19.99113  19.859999
ENSMUSG00000004266  15.910300  16.200255  15.811418   16.45706  15.634751
ENSMUSG00000004267   9.857861  10.760192   9.866651    9.85394   9.864916
ENSMUSG00000004268  18.505564  18.127171  17.997450   18.17507  17.988656
ENSMUSG00000004270  19.289992  19.415306  19.331562   19.17210  19.192401
ENSMUSG00000004285  17.615189  17.179544  17.227645   17.18348  17.636562
ENSMUSG00000004296   9.857861   9.866686   9.866651    9.85394   9.864916
ENSMUSG00000004317  14.593113  13.983481  14.049127   14.18386  13.784273
ENSMUSG00000004319  18.015150  18.230367  18.289361   18.05324  18.105640
ENSMUSG00000004328  10.715539  10.760192  11.005395    9.85394  10.993925
ENSMUSG00000004341  11.367787  11.8378

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

[1] 2


In [18]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train030.Rds",sep=""))

In [19]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

$Fold01
 [1]  11  14  17  29  34  39  41  42  51  60  70 106 111 114 121 122 148 154 162
[20] 181 184 190

$Fold02
 [1]   4  13  18  24  37  53  55  67  71  94 107 110 113 135 137 145 158 164 196
[20] 197 207 215

$Fold03
 [1]  16  32  52  62  68  86 100 103 104 115 124 126 128 140 156 167 169 176 177
[20] 194 202 203

$Fold04
 [1]  20  21  23  46  54  56  63  66  69  73  97 101 125 142 146 149 159 168 175
[20] 195 212 214

$Fold05
 [1]   3  12  38  44  57  77  85  91 102 116 129 130 132 138 143 144 163 165 178
[20] 182 186 210

$Fold06
 [1]   2   6  10  26  35  50  59  83  98 109 112 151 174 189 192 193 200 201 205
[20] 209 211 220

$Fold07
 [1]   1   5  31  36  47  61  64  74  80  84 117 123 134 139 152 157 161 166 180
[20] 191 204 217

$Fold08
 [1]   9  22  28  33  45  72  75  78  87  92  93 105 127 150 153 173 185 187 199
[20] 213 216 219

$Fold09
 [1]   7  25  30  40  49  76  79  88  89  99 108 118 131 147 160 171 172 183 188
[20] 198 208 218

$Fold10
 [1]   8  15  19  27  43  48 

In [20]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

[1]  0 30


In [21]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [22]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"030.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] 9825
[1] "Processed 100 of 9825 reactions (1%)..."
[1] "Processed 200 of 9825 reactions (2%)..."
[1] "Processed 300 of 9825 reactions (3.1%)..."
[1] "Processed 400 of 9825 reactions (4.1%)..."
[1] "Processed 500 of 9825 reactions (5.1%)..."
[1] "Processed 600 of 9825 reactions (6.1%)..."
[1] "Processed 700 of 9825 reactions (7.1%)..."
[1] "Processed 800 of 9825 reactions (8.2%)..."
[1] "Processed 900 of 9825 reactions (9.2%)..."
[1] "Processed 1000 of 9825 reactions (10.2%)..."
[1] "Storing PCA objects containing reactions 0-1000 of 9825 reactions (10.2%)..."
[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 re

In [23]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls030.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls030.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6949822 371.2   10062812 537.5 10062812 537.5
Vcells 20880451 159.4   34054847 259.9 33206179 253.4


In [24]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls030.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train030.Rds", sep = ""))

In [25]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
  print(cur_cv_fold_indices)
  print("----------")
}

 [1]  11  14  17  29  34  39  41  42  51  60  70 106 111 114 121 122 148 154 162
[20] 181 184 190
[1] "----------"
 [1]   4  13  18  24  37  53  55  67  71  94 107 110 113 135 137 145 158 164 196
[20] 197 207 215
[1] "----------"
 [1]  16  32  52  62  68  86 100 103 104 115 124 126 128 140 156 167 169 176 177
[20] 194 202 203
[1] "----------"
 [1]  20  21  23  46  54  56  63  66  69  73  97 101 125 142 146 149 159 168 175
[20] 195 212 214
[1] "----------"
 [1]   3  12  38  44  57  77  85  91 102 116 129 130 132 138 143 144 163 165 178
[20] 182 186 210
[1] "----------"
 [1]   2   6  10  26  35  50  59  83  98 109 112 151 174 189 192 193 200 201 205
[20] 209 211 220
[1] "----------"
 [1]   1   5  31  36  47  61  64  74  80  84 117 123 134 139 152 157 161 166 180
[20] 191 204 217
[1] "----------"
 [1]   9  22  28  33  45  72  75  78  87  92  93 105 127 150 153 173 185 187 199
[20] 213 216 219
[1] "----------"
 [1]   7  25  30  40  49  76  79  88  89  99 108 118 131 147 160 171 172 183 188

In [26]:
%%R
formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00"  "30.00"
[1] "0.00"
[1] "30.00"


In [27]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)

         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  1 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  0.696572080390183 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  0.830136111345493 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.289054721106074 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.015066920871032 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.00692594618614378 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last RXN_ID =  R-M

[1] "Last RXN_ID =  R-MMU-3928599 : Last ARI =  0.182212457556995 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928625 : Last ARI =  0.711301255167761 : Last ECOUNT =  17 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928657 : Last ARI =  0.439472322246317 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9034539 : Last ARI =  0.0492286323180665 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-983156 : Last ARI =  1 : Last ECOUNT =  242 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  0.810108719557322 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] "Last RXN_ID =  R-

[1] "Last RXN_ID =  R-MMU-2426496 : Last ARI =  0.574455895198148 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.8 % remaining..."
[1] "Last RXN_ID =  R-NUL-2467092 : Last ARI =  0.499627368617227 : Last ECOUNT =  7 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-168162 : Last ARI =  0.281295133740846 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-445079 : Last ARI =  0.489713925652509 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8856808 : Last ARI =  0.910699694504766 : Last ECOUNT =  19 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.4 % remaining..."
[1] "Last RXN_ID =  R-MMU-5607725 : Last ARI =  0.757239640827888 : Last ECOUNT =  14 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.3 % remaining..."
[1] "Last

[1] "Last RXN_ID =  R-MMU-2404195 : Last ARI =  0.394859637366238 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-74707 : Last ARI =  0.456588120524622 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-74742 : Last ARI =  0.321369851757574 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-76397 : Last ARI =  0.709363114074165 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8952625 : Last ARI =  0.726061158395652 : Last ECOUNT =  26 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9734336 : Last ARI =  0.408806329537434 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  69.4 % remaining..."
[1] "Last RXN_

[1] "Last RXN_ID =  R-MMU-162857 : Last ARI =  0.0835560234997334 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  59.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9686524 : Last ARI =  0.504897148049218 : Last ECOUNT =  8 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  59.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-198440 : Last ARI =  0.202519820434218 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  59.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6805399 : Last ARI =  0.686128748797815 : Last ECOUNT =  19 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  59.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9832078 : Last ARI =  0.0763649632765108 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  59.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8978970 : Last ARI =  0.539688194637634 : Last ECOUNT =  7 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  59.4 % remaining..."
[1] "Last

[1] "Last RXN_ID =  R-MMU-1810473 : Last ARI =  0.103393086937664 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  49.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-177924 : Last ARI =  0.722677470922478 : Last ECOUNT =  11 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  49.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-177938 : Last ARI =  0.714890794992296 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  49.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-183067 : Last ARI =  0.668712189907737 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  49.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-8864029 : Last ARI =  0.326274948151864 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  49.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9667456 : Last ARI =  0.461723988685428 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  49.4 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-MMU-2468039 : Last ARI =  0.527042556306668 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  39.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-8851234 : Last ARI =  -0.000746608439908102 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  39.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-442387 : Last ARI =  0.522614068544362 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  39.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-70997 : Last ARI =  0.429963964803324 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  39.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-6784204 : Last ARI =  0.183267246356931 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  39.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-9820839 : Last ARI =  0.660018086972604 : Last ECOUNT =  8 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  39.4 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-71802 : Last ARI =  0.507942833555912 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  30 % remaining..."
[1] "Last RXN_ID =  R-MMU-4754181 : Last ARI =  0.447950548743336 : Last ECOUNT =  16 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  29.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-174273 : Last ARI =  0.358159365108528 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  29.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-9644330 : Last ARI =  0.74057267504378 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  29.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-77325 : Last ARI =  0.146487986001998 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  29.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9027321 : Last ARI =  0.300644289388443 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  29.5 % remaining..."
[1] "Last RXN_ID 

[1] "Last RXN_ID =  R-MMU-437092 : Last ARI =  0.21792680504052 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  20 % remaining..."
[1] "Last RXN_ID =  R-MMU-173597 : Last ARI =  0.749335597297479 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5663185 : Last ARI =  0.295407229201289 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-111285 : Last ARI =  0.0275547890170538 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9758661 : Last ARI =  1 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-433089 : Last ARI =  0.00543446347923841 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-2646

[1] "Last RXN_ID =  R-MMU-110352 : Last ARI =  0.159650898438701 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  10 % remaining..."
[1] "Last RXN_ID =  R-MMU-975916 : Last ARI =  0.112895987215927 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-417858 : Last ARI =  0.0989207890268465 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-446214 : Last ARI =  0.051484112568919 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5578712 : Last ARI =  0.177866934945684 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-352052 : Last ARI =  0.162635884852387 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.5 % remaining..."
[1] "Last RXN_ID =  R

In [28]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls030.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls030.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls030.Rds", sep = ""))

In [29]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df030.Rds", sep = ""))

                           RXN_ID MISCLASS           ARI ECOUNT
R-MMU-170666.0.00    R-MMU-170666        0  6.799599e-01     14
R-MMU-170666.30.00   R-MMU-170671        0  6.799599e-01     14
R-MMU-170671.0.00    R-MMU-170674        0  6.799599e-01     14
R-MMU-170671.30.00   R-MMU-170686        0  6.799599e-01     14
R-MMU-170674.0.00    R-MMU-380073        0  1.000000e+00    182
R-MMU-170674.30.00   R-MMU-392187        0  6.361724e-01     22
R-MMU-170686.0.00    R-MMU-392195        0  6.361724e-01     22
R-MMU-170686.30.00   R-MMU-392202        0  6.361724e-01     22
R-MMU-380073.0.00    R-MMU-749454        0  1.000000e+00    178
R-MMU-380073.30.00   R-MMU-749456        0  1.000000e+00    178
R-MMU-392187.0.00    R-MMU-751001        0  6.436117e-01     21
R-MMU-392187.30.00  R-MMU-8964229        0  3.026465e-01      5
R-MMU-392195.0.00   R-MMU-8964242        0  3.026465e-01      5
R-MMU-392195.30.00  R-MMU-8964329        0  3.026465e-01      5
R-MMU-392202.0.00   R-MMU-8982012       

R-MMU-5654224.30.00 R-MMU-9011082        0  3.199526e-01      2
R-MMU-5654392.0.00  R-MMU-9011144        0  1.158329e-02      3
R-MMU-5654392.30.00 R-MMU-9011146        0  1.158329e-02      3
R-MMU-5654397.0.00  R-MMU-9753028        0  3.275507e-01      5
R-MMU-5654397.30.00 R-MMU-9753126        0  3.275507e-01      5
R-MMU-5654399.0.00   R-MMU-514604        0  1.293288e-01      2
R-MMU-5654399.30.00  R-MMU-975389        0  1.990332e-01      4
R-MMU-5654402.0.00  R-MMU-1169394        0  5.700361e-01      5
R-MMU-5654402.30.00 R-MMU-1169398        0  6.214064e-01      5
R-MMU-5654404.0.00  R-MMU-1169402        0  5.700819e-01      6
R-MMU-5654404.30.00 R-MMU-1169403        0  4.550408e-01      4
R-MMU-5654406.0.00  R-MMU-1169405        0  5.880430e-01      5
R-MMU-5654406.30.00 R-MMU-1169406        0  8.378372e-01     12
R-MMU-5654407.0.00  R-MMU-1678842        0  8.613629e-01     13
R-MMU-5654407.30.00 R-MMU-1678843        0  6.903632e-01      5
R-MMU-5654408.0.00  R-MMU-5653754       

R-MMU-9011082.30.00 R-MMU-9692139        0  7.287290e-02      2
R-MMU-9011144.0.00  R-MMU-9692145        0  2.471637e-01      2
R-MMU-9011144.30.00 R-MMU-9692156        0  1.491983e-01      4
R-MMU-9011146.0.00  R-MMU-9692157        0  1.491983e-01      4
R-MMU-9011146.30.00 R-MMU-9692268        0  1.260283e-01      2
R-MMU-9753028.0.00   R-MMU-450434        0  9.676307e-02      1
R-MMU-9753028.30.00  R-MMU-450466        0  9.289153e-01     45
R-MMU-9753126.0.00   R-MMU-450551        0  5.952710e-01      7
R-MMU-9753126.30.00  R-MMU-450580        0  8.482462e-01     12
R-MMU-514604.0.00     R-MMU-72103        0  8.614202e-01     43
R-MMU-514604.30.00  R-MMU-9770119        0  8.924841e-01     69
R-MMU-975389.0.00   R-MMU-9770141        0  9.124775e-01    123
R-MMU-975389.30.00  R-MMU-9770142        0  9.817844e-01    132
R-MMU-1169394.0.00  R-MMU-9770145        0  9.635688e-01    145
R-MMU-1169394.30.00 R-MMU-9770236        0  9.124775e-01    119
R-MMU-1169398.0.00  R-MMU-6798748       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 R-MMU-1358791        0  4.183199e-01      4
R-MMU-5693542.0.00  R-MMU-1445150        0  4.546824e-01      3
R-MMU-5693542.30.00  R-MMU-162352        0  2.726044e-02      1
R-MMU-5693564.0.00   R-MMU-198599        0  3.905395e-01      4
R-MMU-5693564.30.00  R-MMU-198609        0  2.219262e-01      4
R-MMU-5693580.0.00   R-MMU-198611        0  5.178630e-01      4
R-MMU-5693580.30.00  R-MMU-198613        0  7.588886e-01      5
R-MMU-5693593.0.00   R-MMU-198621        0  3.326572e-01      4
R-MMU-5693593.30.00  R-MMU-198640        0  7.791361e-01     10
R-MMU-6782138.0.00   R-MMU-199298        0  2.402172e-01      4
R-MMU-6782138.30.00  R-MMU-199299        0  6.230253e-01      7
R-MMU-6782141.0.00   R-MMU-199425        0  5.442939e-01      5
R-MMU-6782141.30.00  R-MMU-199443        0  6.073294e-01      5
R-MMU-6782204.0.00   R-MMU-199839        0  2.941432e-01      4
R-MMU-6782204.30.00  R-MMU-199863        0  2.678454e-01      4
R-MMU-6782208.0.00   R-MMU-200143        0  4.434117e-01   

...

 R-MMU-8868066        0 -2.017665e-03      1
R-MMU-964773.30.00   R-MMU-196417        0  1.566729e-01      1
R-MMU-9013198.0.00  R-MMU-6807064        0  1.566729e-01      1
R-MMU-9013198.30.00 R-MMU-9616883        0  3.610217e-01      2
R-MMU-428231.0.00   R-MMU-9626284        0  3.018419e-01      1
R-MMU-428231.30.00  R-MMU-9634434        0  3.610217e-01      2
R-MMU-388855.0.00    R-MMU-390909        0  1.486719e-01      1
R-MMU-388855.30.00  R-MMU-9692738        0  1.486719e-01      1
R-MMU-9678854.0.00   R-MMU-156930        0 -6.404243e-03      1
R-MMU-9678854.30.00 R-MMU-5358475        0  5.232805e-02      2
R-MMU-9690449.0.00  R-MMU-9709179        0  1.008522e-03      1
R-MMU-9690449.30.00  R-MMU-109862        0  3.416139e-02      2
R-MMU-5651654.0.00   R-MMU-109864        0  3.416139e-02      2
R-MMU-5651654.30.00 R-MMU-6784652        0  1.322603e-01      1
R-MMU-5651657.0.00  R-MMU-8874201        0  1.429354e-01      2
R-MMU-5651657.30.00 R-MMU-8874202        0  1.322603e-01   

R-MMU-1483159.0.00  R-MMU-9614271        0 -2.459623e-02      1
R-MMU-1483159.30.00 R-MMU-9614273        0 -2.459623e-02      1
R-MMU-417842.0.00   R-MMU-8877902        0  2.659414e-01      2
R-MMU-417842.30.00  R-MMU-8985009        0  3.511079e-02      2
R-MMU-429786.0.00   R-MMU-8985018        0  5.609700e-02      3
R-MMU-429786.30.00  R-MMU-8985315        0  1.980628e-01      2
R-MMU-190782.0.00   R-MMU-8986307        0  2.308744e-01      2
R-MMU-190782.30.00  R-MMU-9008004        0  1.974582e-01      1
R-MMU-191072.0.00   R-MMU-9008009        0  1.974582e-01      1
R-MMU-191072.30.00  R-MMU-9009309        0  2.076327e-01      2
R-MMU-375340.0.00   R-NUL-8939977        0  1.974582e-01      1
R-MMU-375340.30.00  R-NUL-8985474        0  1.974582e-01      1
R-MMU-174959.0.00   R-NUL-8985490        0  1.974582e-01      1
R-MMU-174959.30.00  R-NUL-9008555        0  1.974582e-01      1
R-MMU-174963.0.00   R-NUL-9008636        0  1.974582e-01      1
R-MMU-174963.30.00  R-NUL-9009236       

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



      1
R-MMU-428273.30.00    R-MMU-70941        0  1.895307e-01      1
R-MMU-9836621.0.00   R-MMU-374207        0  5.497376e-02      1
R-MMU-9836621.30.00 R-MMU-8980228        0 -1.625545e-02      1
R-MMU-428779.0.00   R-MMU-6787533        0 -8.927672e-03      1
R-MMU-428779.30.00  R-MMU-4084989        0  2.217719e-01      1
R-MMU-8987120.0.00  R-MMU-9708104        0 -1.931167e-02      1
R-MMU-8987120.30.00 R-MMU-5688294        0  4.607711e-01      3
R-MMU-8937442.0.00  R-MMU-8850797        0  2.139625e-01      1
R-MMU-8937442.30.00  R-MMU-444204        0  1.760798e-02      1
R-MMU-1254376.0.00  R-MMU-9842656        0  2.088523e-01      3
R-MMU-1254376.30.00 R-MMU-9011343        0  1.618827e-02      1
R-MMU-211923.0.00   R-MMU-9816416        0  2.039033e-01      1
R-MMU-211923.30.00  R-MMU-9820700        0  1.924337e-01      4
R-MMU-189423.0.00   R-MMU-1296035        0  6.594479e-02      1
R-MMU-189423.30.00  R-MMU-9013021        0  2.400261e-01      2
R-MMU-6786421.0.00  R-MMU-969049

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [30]:
%%R
print(dim(vst.count.mtx.train))

[1] 9297  220


In [31]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id030.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id030.txt",sep=""), header=FALSE)
print(sid)

            V1
1   SRR7817611
2   SRR7817612
3   SRR7817614
4   SRR7817615
5   SRR7817616
6   SRR7817617
7   SRR7817618
8   SRR7817619
9   SRR7817620
10  SRR7817621
11  SRR7817622
12  SRR7817623
13  SRR7817624
14  SRR7817625
15  SRR7817626
16  SRR7817627
17  SRR7817628
18  SRR7817629
19  SRR7817630
20  SRR7817631
21  SRR7817632
22  SRR7817633
23  SRR7817634
24  SRR7817635
25  SRR7817636
26  SRR7817637
27  SRR7817638
28  SRR7817639
29  SRR7817640
30  SRR7817641
31  SRR7817642
32  SRR7817643
33  SRR7817644
34  SRR7817645
35  SRR7817646
36  SRR7817647
37  SRR7817648
38  SRR7817649
39  SRR7817650
40  SRR7817651
41  SRR7817652
42  SRR7817653
43  SRR7817654
44  SRR7817655
45  SRR7817656
46  SRR7817657
47  SRR7817658
48  SRR7817659
49  SRR7817660
50  SRR7817661
51  SRR7817662
52  SRR7817663
53  SRR7817666
54  SRR7817667
55  SRR7817668
56  SRR7817669
57  SRR7817670
58  SRR7817671
59  SRR7817672
60  SRR7817673
61  SRR7817674
62  SRR7817675
63  SRR7817676
64  SRR7817677
65  SRR7817678
66  SRR781

In [32]:
%%R
print(unique(tcdd_dose_detail.vec.train))

[1]  0 30


In [33]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  
   scale_colour_manual(values = c("0" = "red", "30" = "green")) +  
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  
     colors = c("red", "green"),  
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  
)
saveWidget(p, "plotly_chart030.html", selfcontained = TRUE)